In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0] 
    # '0,1,2,3,4,5'
    print(os.environ['CUDA_VISIBLE_DEVICES'])
# !nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


3


In [7]:
import argparse
import json
import pickle
import string
import time
import matplotlib.pyplot as plt

import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sklearn.cluster import KMeans, AgglomerativeClustering
import torch
from tqdm import tqdm

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

from note_pruning_analysis import get_lm_output, get_dataset, convert_example_to_str

In [8]:
model_name = 'all-mpnet-base-v2'
model_name_or_path = 'results/baselines/sentence-transformers/all-mpnet-base-v2'

dataset = 'wizardlm'
input_file = 'data/processed/sharegpt/sharegpt_data.jsonl'
input_file = 'data/processed/ultrachat/ultrachat_data.jsonl'
input_file = 'data/processed/wizardlm/wizardlm_data.jsonl'
n_clusters = 50
batch_size = 128

encode_fn_type = 'input'

In [9]:

model = AutoModel.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


[2023-11-12 21:01:55,077] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [10]:
ds = get_dataset(dataset, processed=True)
ds

Found cached dataset json (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/wizardlm/json/default-a3f71ae376e517fb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


Dataset({
    features: ['dataset', 'id', 'messages'],
    num_rows: 143000
})

In [11]:
d = get_lm_output(dataset, 
                  model_name, 
                  encode_fn_type=encode_fn_type,
                  return_text_embedding=True,)

In [19]:
embeddings = d['text_embedding']
embeddings = embeddings / np.linalg.norm(embeddings, axis=-1, keepdims=True)
embeddings = embeddings[:10_000]

In [47]:
# from sklearn.cluster import KMeans, MiniBatchKMeans


# kmeans = MiniBatchKMeans(
#     n_clusters=n_clusters, 
#     init='k-means++',
#     n_init='auto',
#     random_state=0,
#     verbose=True,
#     batch_size=128,
#     max_no_improvement=100,
#     reassignment_ratio=1e-4,
# )

# kmeans.fit(embeddings)



Init 1/1 with method k-means++
Inertia for init 1/1: 515.49755859375
[MiniBatchKMeans] Reassigning 12 cluster centers.
Minibatch step 1/7812: mean batch inertia: 1.3069249391555786
Minibatch step 2/7812: mean batch inertia: 0.9298588633537292, ewa inertia: 0.9298588633537292
Minibatch step 3/7812: mean batch inertia: 0.8834523558616638, ewa inertia: 0.9286709755507127
Minibatch step 4/7812: mean batch inertia: 0.8550400137901306, ewa inertia: 0.9267862114060561
Minibatch step 5/7812: mean batch inertia: 0.8502101302146912, ewa inertia: 0.9248260597427235
Minibatch step 6/7812: mean batch inertia: 0.864001452922821, ewa inertia: 0.9232691055035579
Minibatch step 7/7812: mean batch inertia: 0.8516941666603088, ewa inertia: 0.9214369702826928
Minibatch step 8/7812: mean batch inertia: 0.8346852660179138, ewa inertia: 0.919216348715671
Minibatch step 9/7812: mean batch inertia: 0.842236340045929, ewa inertia: 0.9172458575428429
Minibatch step 10/7812: mean batch inertia: 0.8253653049468994

Minibatch step 88/7812: mean batch inertia: 0.8110902905464172, ewa inertia: 0.8195055597778264
Minibatch step 89/7812: mean batch inertia: 0.8039036393165588, ewa inertia: 0.8191061905509406
Minibatch step 90/7812: mean batch inertia: 0.8058323264122009, ewa inertia: 0.8187664136066833
Minibatch step 91/7812: mean batch inertia: 0.7855474352836609, ewa inertia: 0.8179160927936953
Minibatch step 92/7812: mean batch inertia: 0.8188366889953613, ewa inertia: 0.8179396576999672
Minibatch step 93/7812: mean batch inertia: 0.7897253036499023, ewa inertia: 0.8172174424578097
Minibatch step 94/7812: mean batch inertia: 0.7957373261451721, ewa inertia: 0.8166676064638057
Minibatch step 95/7812: mean batch inertia: 0.8179012537002563, ewa inertia: 0.8166991846752376
Minibatch step 96/7812: mean batch inertia: 0.792192280292511, ewa inertia: 0.8160718706544419
Minibatch step 97/7812: mean batch inertia: 0.8000697493553162, ewa inertia: 0.8156622573105186
Minibatch step 98/7812: mean batch inerti

Minibatch step 181/7812: mean batch inertia: 0.7916180491447449, ewa inertia: 0.8024026907424225
Minibatch step 182/7812: mean batch inertia: 0.7898751497268677, ewa inertia: 0.8020820177597225
Minibatch step 183/7812: mean batch inertia: 0.8005515336990356, ewa inertia: 0.8020428412854164
Minibatch step 184/7812: mean batch inertia: 0.8165047764778137, ewa inertia: 0.8024130298074895
Minibatch step 185/7812: mean batch inertia: 0.7948412299156189, ewa inertia: 0.8022192111121271
Minibatch step 186/7812: mean batch inertia: 0.7934420704841614, ewa inertia: 0.8019945387792845
Minibatch step 187/7812: mean batch inertia: 0.8063995838165283, ewa inertia: 0.8021072966564502
Minibatch step 188/7812: mean batch inertia: 0.8016238212585449, ewa inertia: 0.8020949209238372
Minibatch step 189/7812: mean batch inertia: 0.7941296100616455, ewa inertia: 0.801891029354922
Minibatch step 190/7812: mean batch inertia: 0.8190851211547852, ewa inertia: 0.8023311540925246
Minibatch step 191/7812: mean b

Minibatch step 280/7812: mean batch inertia: 0.7945882678031921, ewa inertia: 0.7981250417349918
Minibatch step 281/7812: mean batch inertia: 0.7885631322860718, ewa inertia: 0.79788028132914
Minibatch step 282/7812: mean batch inertia: 0.8018405437469482, ewa inertia: 0.7979816539097778
Minibatch step 283/7812: mean batch inertia: 0.8066378235816956, ewa inertia: 0.7982032296958004
Minibatch step 284/7812: mean batch inertia: 0.7947407364845276, ewa inertia: 0.7981145987326881
Minibatch step 285/7812: mean batch inertia: 0.7849825620651245, ewa inertia: 0.7977784522086508
Minibatch step 286/7812: mean batch inertia: 0.801082193851471, ewa inertia: 0.797863019537974
Minibatch step 287/7812: mean batch inertia: 0.7907319068908691, ewa inertia: 0.797680481308031
Minibatch step 288/7812: mean batch inertia: 0.7965618968009949, ewa inertia: 0.7976518484079409
Minibatch step 289/7812: mean batch inertia: 0.8077215552330017, ewa inertia: 0.7979096071267906
Minibatch step 290/7812: mean batch

Minibatch step 379/7812: mean batch inertia: 0.8001161813735962, ewa inertia: 0.7955717810999712
Minibatch step 380/7812: mean batch inertia: 0.8036395907402039, ewa inertia: 0.7957782963752336
Minibatch step 381/7812: mean batch inertia: 0.7800031900405884, ewa inertia: 0.7953744940333008
Minibatch step 382/7812: mean batch inertia: 0.79486483335495, ewa inertia: 0.795361448024536
Minibatch step 383/7812: mean batch inertia: 0.7953992486000061, ewa inertia: 0.7953624156225082
Minibatch step 384/7812: mean batch inertia: 0.7949291467666626, ewa inertia: 0.7953513250488559
Minibatch step 385/7812: mean batch inertia: 0.8018376231193542, ewa inertia: 0.7955173576761979
Minibatch step 386/7812: mean batch inertia: 0.7879263758659363, ewa inertia: 0.7953230479728255
Minibatch step 387/7812: mean batch inertia: 0.7943538427352905, ewa inertia: 0.7952982387996619
Minibatch step 388/7812: mean batch inertia: 0.7914602756500244, ewa inertia: 0.7951999967672344
Minibatch step 389/7812: mean bat

Minibatch step 517/7812: mean batch inertia: 0.7980895638465881, ewa inertia: 0.79787072717757
Minibatch step 518/7812: mean batch inertia: 0.7967276573181152, ewa inertia: 0.7978414675151342
Minibatch step 519/7812: mean batch inertia: 0.7829297780990601, ewa inertia: 0.7974597664361905
Minibatch step 520/7812: mean batch inertia: 0.7726361751556396, ewa inertia: 0.7968243460414479
Minibatch step 521/7812: mean batch inertia: 0.8016061782836914, ewa inertia: 0.7969467487065828
Minibatch step 522/7812: mean batch inertia: 0.7888655066490173, ewa inertia: 0.7967398895958202
Minibatch step 523/7812: mean batch inertia: 0.782885730266571, ewa inertia: 0.796385258580093
Minibatch step 524/7812: mean batch inertia: 0.8090127110481262, ewa inertia: 0.7967084890402286
Minibatch step 525/7812: mean batch inertia: 0.7922223210334778, ewa inertia: 0.7965936546226975
Minibatch step 526/7812: mean batch inertia: 0.804695188999176, ewa inertia: 0.796801033164881
Minibatch step 527/7812: mean batch 

MiniBatchKMeans(batch_size=128, max_no_improvement=100, n_clusters=50,
                n_init='auto', random_state=0, reassignment_ratio=0.0001,
                verbose=True)

In [20]:
## kmeans 

clustering_model = KMeans(
    n_clusters=n_clusters, 
    n_init="auto",
    random_state=0,
    verbose=True)
clustering_model.fit(embeddings)


centers = torch.from_numpy(clustering_model.cluster_centers_)
labels = torch.from_numpy(clustering_model.labels_)

# Sort labels
classes, counts = np.unique(labels, return_counts=True)
indices = np.argsort(counts)[::-1]
classes = [classes[i] for i in indices]
new_labels = torch.empty_like(labels)
new_centers = torch.empty_like(centers)
for i, c in enumerate(classes):
    new_labels[labels == c] = i
    new_centers[i] = centers[c]
    
labels = new_labels
centers = new_centers


Initialization complete
Iteration 0, inertia 13046.0478515625.
Iteration 1, inertia 7963.15087890625.
Iteration 2, inertia 7823.31982421875.
Iteration 3, inertia 7766.98583984375.
Iteration 4, inertia 7734.853515625.
Iteration 5, inertia 7714.99462890625.
Iteration 6, inertia 7701.0830078125.
Iteration 7, inertia 7689.73876953125.
Iteration 8, inertia 7681.1552734375.
Iteration 9, inertia 7675.09716796875.
Iteration 10, inertia 7670.42822265625.
Iteration 11, inertia 7666.07763671875.
Iteration 12, inertia 7661.99072265625.
Iteration 13, inertia 7658.1611328125.
Iteration 14, inertia 7654.46142578125.
Iteration 15, inertia 7651.60595703125.
Iteration 16, inertia 7649.0810546875.
Iteration 17, inertia 7646.892578125.
Iteration 18, inertia 7645.1513671875.
Iteration 19, inertia 7643.607421875.
Iteration 20, inertia 7642.076171875.
Iteration 21, inertia 7640.2158203125.
Iteration 22, inertia 7638.42626953125.
Iteration 23, inertia 7636.91845703125.
Iteration 24, inertia 7635.806640625.
It

In [40]:
from fastchat_topic_clustering import get_topk_indices, print_topk

texts = [x[0]['content'] for x in ds[:len(labels)]['messages']]

k = 200
show_cut_off = 500
topk_indices = get_topk_indices(centers, labels, embeddings, k)
topk_str = print_topk(texts, labels, topk_indices, show_cut_off)
num_clusters = len(centers)

print(topk_indices.shape, len(topk_str))

torch.Size([50, 86]) 1747637


In [61]:
# c_minibatchkmeans = c
# c_kmeans = c

for i in range(len(c)):
    print(i, c_kmeans[i][1], c_minibatchkmeans[i][1])

TypeError: object of type 'numpy.int32' has no len()

In [62]:
from collections import Counter
c = dict(Counter(labels.tolist()))
c = sorted(c.items())
c

# kmeans
# [(0, 142),
#  (1, 136),
#  (2, 116),
#  (3, 112),
#  (4, 109),
#  (5, 89),
#  (6, 82),
#  (7, 73),
#  (8, 73),
#  (9, 68)]


[(0, 5024),
 (1, 3236),
 (2, 3184),
 (3, 2815),
 (4, 2656),
 (5, 2558),
 (6, 2515),
 (7, 2327),
 (8, 2323),
 (9, 2316),
 (10, 2308),
 (11, 2304),
 (12, 2289),
 (13, 2232),
 (14, 2182),
 (15, 2171),
 (16, 2159),
 (17, 2152),
 (18, 2150),
 (19, 2128),
 (20, 2115),
 (21, 2109),
 (22, 2097),
 (23, 2085),
 (24, 2079),
 (25, 1984),
 (26, 1880),
 (27, 1865),
 (28, 1842),
 (29, 1807),
 (30, 1804),
 (31, 1797),
 (32, 1760),
 (33, 1742),
 (34, 1740),
 (35, 1700),
 (36, 1689),
 (37, 1666),
 (38, 1638),
 (39, 1579),
 (40, 1532),
 (41, 1402),
 (42, 1398),
 (43, 1368),
 (44, 1266),
 (45, 1238),
 (46, 1188),
 (47, 1164),
 (48, 987),
 (49, 450)]

In [64]:
print(topk_str)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



tensor([False, False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False,  True, False, False, False,
         True, False, False, False, False, False, False, False, False, False,
         True, False,  True, False, False, False, False,  True,  True, False,
         True, False, False, False, False, False, False, False, False, False,
         True, False, False, False, False,  True,  True, False, False, False,
        False, False, False,  True, False, False,  True, False, False,  True,
         True, False, False, False, False, False, False, False, False, False,
        False, False,  True,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [22]:
from fastchat_topic_clustering import get_cluster_info

cluster_alg = 'kmeans'
# Dump results
filename_prefix = f"results/clustering/c{n_clusters}_{cluster_alg}"
os.makedirs(os.path.dirname(filename_prefix), exist_ok=True)

texts_np = np.array(texts)

with open(filename_prefix + "_topk.txt", "w") as fout:
    fout.write(topk_str)

with open(filename_prefix + "_all.jsonl", "w") as fout:
    for i in range(len(centers)):
        tmp_indices = labels == i
        tmp_embeddings = embeddings[tmp_indices]
        tmp_texts = texts_np[tmp_indices]

        scores = cos_sim(centers[i].unsqueeze(0), tmp_embeddings)[0]
        sorted_indices = torch.flip(torch.argsort(scores), dims=[0])

        for text, score in zip(tmp_texts[sorted_indices], scores[sorted_indices]):
            obj = {"cluster": i, "text": text, "sim": score.item()}
            fout.write(json.dumps(obj, ensure_ascii=False) + "\n")

# cluster_info = get_cluster_info(texts, labels, topk_indices)
# with open(filename_prefix + "_cluster.json", "w") as fout:
#     json.dump(cluster_info, fout, ensure_ascii=False)

KeyboardInterrupt: 

In [23]:

cluster_info = []
for k in range(len(topk_indices)):
    num_samples = torch.sum(labels == k).item()
    topk_prompts = []
    for idx in topk_indices[k]:
        topk_prompts.append(texts[idx])
    random_prompts = []
    for idx in range(len(topk_indices)):
        random_prompts.append(np.random.choice(texts))
    cluster_info.append((num_samples, topk_prompts, random_prompts))
    break

In [38]:

# num_samples, len(topk_prompts)


topk_indices.shape



torch.Size([50, 86])

In [12]:

i = 12


t = dsf[i]['messages'][0]['content']
print(convert_example_to_str(0, dsf[i]))
print(len(t))
print(t)

metadata: {
    "dataset": "ultrachat",
    "id": "ultrachat_113",
    "idx": 0,
    "n_turns": 8
}

= USER        =
Here is a piece of text: DUBAI, Aug 26 (Reuters) - Saudi Arabia’s stock market outperformed by a large margin on Sunday, led by petrochemical stocks, as Middle Eastern bourses reopened after a one-week break for the Muslim festival of Eid Al Adha.
The Saudi stock index rose 1.4 percent, although trading volume was the lowest this year as some individual investors had not yet returned from holidays. Gainers outnumbered losers by 146 to 23.
The market has been buoyed this year by inflows of foreign funds in anticipation of Riyadh joining emerging market indexes next year. Recent exchange data indicates those inflows have slowed but not halted as stock valuations have risen.
Petrochemical makers were particularly strong after the Brent oil price rebounded above $75.0 a barrel last week. The top petrochemical firm, Saudi Basic Industries , added 1.8 percent and Saudi Kayan r

NameError: name 't' is not defined

In [30]:
print(under_min_length, above_max_length)

664 32991


In [17]:
from fastchat_topic_clustering import get_embeddings


embeddings = get_embeddings(texts, args.model, args.batch_size)

[2023-11-08 00:02:46,673] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Batches:   4%|▍         | 53/1202 [01:36<34:53,  1.82s/it]

KeyboardInterrupt



Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x7f19c4a4e590>> (for post_execute):


KeyboardInterrupt: 

In [16]:
from fastchat_topic_clustering import get_embeddings


embeddings = get_embeddings(texts, args.model, args.batch_size)
if args.cluster_alg == "kmeans":
    centers, labels = run_k_means(embeddings, num_clusters)
elif args.cluster_alg == "aggcls":
    centers, labels = run_agg_cluster(embeddings, num_clusters)
elif args.cluster_alg == "HDBSCAN":
    centers, labels = run_hdbscan_cluster(embeddings)
else:
    raise ValueError(f"Invalid clustering algorithm: {args.cluster_alg}")


307563